In [1]:

%cd /home/teresakim/fungiclef-2024/
from fungiclef.utils import get_spark

# https://knowledge.informatica.com/s/article/000196886?language=en_US
# The vectorized reader will run out of memory (8gb) with the default batch size, so
# this is one way of handling the issue. This is likely due to the fact that the data
# column is so damn big, and treated as binary data instead of something like a string.
# We might also be able to avoid this if we don't cache the fields into memory, but this
# this needs to be validated by hand. 
spark = get_spark(**{
    # "spark.sql.parquet.columnarReaderBatchSize": 512,
    "spark.sql.parquet.enableVectorizedReader": False, 
})

gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/"

df = spark.read.parquet(gcs_parquet_path)
df.printSchema()
df.show(1, vertical=True, truncate=False)
df.count() # 295938

/home/teresakim/fungiclef-2024


/home/fungi/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/29 06:08:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/29 06:08:50 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


root
 |-- image_path: string (nullable = true)
 |-- data: binary (nullable = true)
 |-- observationID: long (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- day: double (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- taxonID: double (nullable = true)
 |-- scientificName: string (nullable = true)
 |-- kingdom: string (nullable = true)
 |-- phylum: string (nullable = true)
 |-- class: string (nullable = true)
 |-- order: string (nullable = true)
 |-- family: string (nullable = true)
 |-- genus: string (nullable = true)
 |-- specificEpithet: string (nullable = true)
 |-- taxonRank: string (nullable = true)
 |-- species: string (nullable = true)
 |-- level0Gid: string (nullable = true)
 |-- level0Name: string (nullable = true)
 |-- level1Gid: string (nullable = true)
 |-- level1Name: string (nullable = true)
 |-- level2Gid: string (nullable = true)
 |-- level2Name: string (nullable = tr

24/02/29 06:09:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

295938

In [2]:
from torchvision import transforms
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
from PIL import Image
import io

# define a UDF to convert binary to PIL image, apply transformations, and return a flattened image tensor
def preprocess_image(data):
    image = Image.open(io.BytesIO(data)).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image_tensor = transform(image)
    # flatten the image tensor to return a 1D array
    return image_tensor.numpy().flatten().tolist()

preprocess_udf = udf(preprocess_image, ArrayType(FloatType()))

# apply UDF
df = df.withColumn("processed_image", preprocess_udf(df["data"]))


/home/fungi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
